In [1]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree

In [ ]:
def get_soup_by_url(url, product: str = None):
    if product is None:
        url = url
    else:
        url = f'{url}/{product}'
    r = requests.get(url=url)
    s = BeautifulSoup(r.content, 'html.parser')
    return s

def get_vehicle_info(url):
    s = get_soup_by_url(url)
    text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
    parts = text.split(' · ')
    location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
    pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
    year = parts[0].split(' | ')[0]
    kilometrage = parts[0].split(' | ')[1].replace('km', '')
    publication_date = parts[1]
    output_dict = {
        "Year": year,
        "Kilometrage": kilometrage,
        "Publication Date": publication_date,
        "Location": location,
        "Pub Number": pub_number,
        "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return output_dict
data = pd.read_csv('final_data.csv')

In [2]:
link = 'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [11]:
link

'https://carro.mercadolibre.com.co/MCO-1379439831-mercedes-benz-gle-350-d-4matic-2017-_JM#position=1&search_layout=grid&type=item&tracking_id=79a64cb3-adcf-4d29-be31-812b12b34cef'

In [3]:
r = requests.get(url=link)

s = BeautifulSoup(r.content, 'html.parser')

In [4]:
# publication number

int(s.find_all('span', attrs= {'class':'andes-money-amount__fraction'})[0].text.replace('.', ''))

189000000

In [5]:
# precio
s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')

'1379439831'

In [16]:
import datetime

text = s.find_all('span', attrs= {'class':'ui-pdp-subtitle'})[0].text.replace('.', '')
parts = text.split(' · ')
location = s.find('p', attrs = {'class':'ui-seller-info__status-info__subtitle'}).text
pub_number = s.find_all('span', attrs= {'class':'ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD'})[0].text.replace('#','')
year = parts[0].split(' | ')[0]
kilometrage = parts[0].split(' | ')[1].replace('km', '')
publication_date = parts[1]
output_dict = {
  "Year": year,
  "Kilometrage": kilometrage,
  "Publication Date": publication_date,
  "Location": location,
  "Pub Number": pub_number,
  "Created At": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

output_dict

{'Year': '2017',
 'Kilometrage': '47800 ',
 'Publication Date': 'Publicado hace 3 días',
 'Location': 'Chicó Navarra - Usaquén - Bogotá D.C.',
 'Pub Number': '1379439831',
 'Created At': '2024-01-13 13:35:22'}

'Chicó Navarra - Usaquén - Bogotá D.C.'

In [8]:
table = s.find_all('table', attrs= {'class': 'andes-table'})

In [9]:
table

[]

In [10]:
rows = table[0].find_all('tr')
rows

IndexError: list index out of range

In [106]:
data = {}
for row in rows:
  cols = row.find_all(['th', 'td'])
  cols = [ele.text.strip() for ele in cols]
  data[cols[0]] = cols[1]


In [107]:
data

{'Marca': 'Mercedes-Benz',
 'Modelo': 'Clase GLE',
 'Año': '2017',
 'Versión': '3.0 Coupe 4matic Diésel',
 'Color': 'Gris',
 'Tipo de combustible': 'Diésel',
 'Puertas': '5',
 'Transmisión': 'Automática',
 'Motor': '3.0',
 'Tipo de carrocería': 'Station Wagon',
 'Kilómetros': '47800 km'}

In [22]:
rows

NameError: name 'rows' is not defined

In [49]:
dom = etree.HTML(str(s))

In [50]:
dom

<Element html at 0x7baa0c6bbfc0>

In [51]:
dom.xpath("//table[@class = 'andes-table']")

[]